<a href="https://colab.research.google.com/github/dfragos/EMOS-2024/blob/main/EMOS_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Εγκατάσταση απαιτούμενων βιβλιοθηκών
# pip install pandas pyarrow numpy pycountry

import pandas as pd
import numpy as np
import pycountry

In [ ]:
# Λήψη δεδομένων Eurostat (παράδειγμα dataset για ανεργία)
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/une_rt_m$defaultview/?format=TSV"
unemployment_data = pd.read_csv(url, sep='\t')

# Προβολή δείγματος δεδομένων
print(unemployment_data.head())

In [ ]:
# Επισκόπηση ελλείπουσων τιμών
missing_summary = unemployment_data.isnull().sum()
print(missing_summary)

# Διαγραφή γραμμών με πολλές ελλείπουσες τιμές
unemployment_data.dropna(inplace=True)

# Συμπλήρωση ελλείπουσων τιμών με μέθοδο παρεμβολής
unemployment_data.interpolate(method='linear', inplace=True)

In [ ]:
# Καθορισμός ορίων για ακραίες τιμές (π.χ., 1.5 IQR)
q1 = unemployment_data['values'].quantile(0.25)
q3 = unemployment_data['values'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Αντικατάσταση ακραίων τιμών με NaN
unemployment_data['values'] = unemployment_data['values'].apply(
    lambda x: np.nan if x < lower_bound or x > upper_bound else x
)

In [ ]:
# Τυποποίηση κατηγορικών τιμών (π.χ., geo, sex, age)
unemployment_data['geo'] = unemployment_data['geo'].str.upper().str.strip()
unemployment_data['sex'] = unemployment_data['sex'].str.capitalize().str.strip()
unemployment_data['age'] = unemployment_data['age'].str.replace(" ", "_")

In [ ]:
# Εντοπισμός και αφαίρεση διπλών εγγραφών
unemployment_data.drop_duplicates(inplace=True)

In [ ]:
# Συμπλήρωση ελλειπουσών τιμών με μέσο όρο ανά κατηγορία
unemployment_data['values'] = unemployment_data.groupby(['geo', 'sex', 'age'])['values'].transform(
    lambda x: x.fillna(x.mean())
)

In [ ]:
# Καθαρισμός ανεπιθύμητων χαρακτήρων στις στήλες
unemployment_data['geo'] = unemployment_data['geo'].str.strip()
unemployment_data['sex'] = unemployment_data['sex'].str.strip()
unemployment_data['age'] = unemployment_data['age'].str.strip()

In [ ]:
# Αφαίρεση αρνητικών και μη λογικών τιμών
unemployment_data = unemployment_data[unemployment_data['values'] >= 0]

In [ ]:
# Μετατροπή τιμών σε σωστές μορφές
unemployment_data['values'] = pd.to_numeric(unemployment_data['values'], errors='coerce')
unemployment_data['time'] = pd.to_datetime(unemployment_data['time'])

In [ ]:
# Φιλτράρισμα δεδομένων με βάση λογικά όρια (0-100 για ποσοστά)
unemployment_data = unemployment_data[(unemployment_data['values'] >= 0) & (unemployment_data['values'] <= 100)]

In [ ]:
# Αποθήκευση των καθαρισμένων δεδομένων σε αρχείο CSV
unemployment_data.to_csv("cleaned_unemployment_data.csv", index=False)

# Έλεγχος αποθήκευσης
loaded_csv_data = pd.read_csv("cleaned_unemployment_data.csv")
print(loaded_csv_data.head())

# Αποθήκευση σε αρχείο Parquet
unemployment_data.to_parquet("cleaned_unemployment_data.parquet", index=False)

# Έλεγχος αποθήκευσης
loaded_data = pd.read_parquet("cleaned_unemployment_data.parquet")
print(loaded_data.head())
